In [ ]:
# Install necessary packages
%pip install bokeh pandas

In [ ]:
# Import packages
from math import pi
import pandas as pd
from bokeh.plotting import show, figure, output_file, output_notebook
from bokeh.models import (Legend, HoverTool, Div)
from bokeh.layouts import Column, Row

output_notebook()

output_file('report.html')

In [ ]:
# Import csv
inFile = "report.csv"

inDataFrame = pd.read_csv(inFile, encoding='utf-16',index_col=False)
# Remove planned expenses
inDataFrame = inDataFrame[~inDataFrame['Date'].isna()]
# Rename uncategorized expenses
inDataFrame[' Category'].fillna('Untitled',inplace=True)
inDataFrame = inDataFrame.fillna(0)
# inDataFrame.head()

In [ ]:
#convert colors
dec2HexColor = lambda x: f'#{hex(int(0xFFFFFFFF)+int(x)+1)[-6:].upper()}'
inDataFrame[' Account Color'] = inDataFrame[' Account Color'].map(dec2HexColor)
inDataFrame[' To Account Color'] = inDataFrame[' To Account Color'].map(dec2HexColor)
inDataFrame[' Category Color'] = inDataFrame[' Category Color'].map(dec2HexColor)

In [ ]:
# Get years
inDataFrame['Date'] = pd.to_datetime(inDataFrame['Date'])
years = inDataFrame['Date'].dt.year.unique()

In [ ]:
def piePlot(title: str, df: pd.Series, colors: list[str], percAggregate: int = 20):

    aggregated = df[~(df[' Type']=="TRANSFER")].groupby([' Category'])[' Amount'].sum()
    aggregated.sort_values(key=abs, inplace=True, ascending=False)
    
    # Categories as a percentage of the total
    perc = aggregated/aggregated.sum()
    
    # select the most important elements
    if len(aggregated) == 1:
        selected = aggregated
    else:
        selected = aggregated[perc.cumsum() < (1-percAggregate/100)]
        # at least one
        if len(selected) < 1:
            selected = aggregated.iloc[0]
        selected.loc["Other"] = aggregated.iloc[len(selected):].sum()

    # also aggregate perc
    perc = selected/selected.sum()
    
    angles = (2*pi*perc.cumsum()).tolist()

    source = selected.reset_index(name='value').rename(columns={' Category': "category"})
    source['start']  = [0] + angles[:-1]
    source['end'] = angles
    source['colors'] = [colors[cat] for cat in selected.index.to_list()]
    
    p = figure(height=350, title=title, toolbar_location=None,
           tools="hover", tooltips="@category: @value{0,0}", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle="start", end_angle="end",
            line_color="white", fill_color="colors", legend_field="category", source=source)
    
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    return(p)

def yearlyPlot(yr: int, df: pd.DataFrame, colors: list[str]):

    dfN = df[df[' Amount'] < 0]
    dfP = df[df[' Amount'] >= 0]

    pN = piePlot(f"Yearly expenses by category ({yr})", dfN, colors, 15)
    pP = piePlot(f"Yearly income by category ({yr})", dfP, colors, 15)
    
    print(yr)
    # print(df.head())
    return [pN, pP]

In [ ]:
def monthlyPlot(yr: int, df: pd.DataFrame, colors: list[str], width: int = 1000):
    source = df.groupby([' Category', df.Date.dt.month])[' Amount'].sum()
    
    # create a new plot with a title and axis labels
    p = figure(title=f"Monthly expenses by category ({yr})",
               x_axis_label="Date", y_axis_label="Expenses",
              height=700, width=width)
    
    p.add_layout(Legend(), 'right')
    # add multiple renderers
    for cat in df[' Category'].unique():
        x = source[cat].index.to_list()
        y = source[cat].to_list()
        l = p.line(x, y, name=cat, legend_label=cat, color=colors[cat], line_width=2)
    p.add_tools(HoverTool(tooltips=[("Category", "$name"), ("Month", "@x{0}"), ("Expense", "@y{0,0}")],
                          mode="vline",
                         point_policy='snap_to_data'))

    p.legend.click_policy="hide"

    # show the results
    return(p)

In [ ]:
def catEntry(df: pd.DataFrame):
    # Generate helper dataframes
    pos = df[df[' Amount']>0].groupby([' Category'])[' Amount'].sum()
    neg = df[df[' Amount']<0].groupby([' Category'])[' Amount'].sum()
    neg.sort_values(key=abs, inplace=True, ascending=False)

    tmp = pd.concat([neg, pos], axis=1).fillna(0)
    tmp.columns = ["neg", "pos"]
    tmp["net"] = tmp.neg+tmp.pos
    
    # reference number for percentage evaluation
    refPos = pos.max()
    refNeg = neg.min()
    refNet = tmp.net.abs().max()

    # table entries for each category
    catEntryStr = ""

    for cat in tmp.index:
        vPos = tmp.loc[cat].pos
        vNeg = tmp.loc[cat].neg
        vNet = tmp.loc[cat].net
        catSmall = cat.replace(" ","").lower()

        catEntryStr += f"""<tr>
    <td>{cat}</td>
    <td><abbr title="{vNeg}"><progress class="progress is-small is-{catSmall} flipped" value="{(100*vNeg/refNeg):.00f}" max="100"></progress></abbr></td>
    <td><abbr title="{vPos}"><progress class="progress is-small is-{catSmall}" value="{(100*vPos/refPos):.00f}" max="100"/></progress></abbr></td>
    <td class="has-text-right">{vNet:.00f}:<progress class="progress is-small is-{catSmall} net {"flipped" if vNet<0 else ""}" value="{abs(100*vNet/refNet):.00f}" max="100"/></progress></td>
    <td>{vNet/12:.00f}</td>
    <td>{vNet/(52):.00f}</td>
</tr>
"""
    return catEntryStr

In [ ]:
# Get category colors
tmp = inDataFrame[[' Category',' Category Color']]
flt = inDataFrame[' Category'].drop_duplicates().index
tmp = tmp.loc[flt].to_dict(orient='split')
colors = dict(tmp['data'])
colors["Other"] = "#1C1C3C"


# Convert colors in css style for progress bars
customColors=""".progress.net {
    display: inline;
    width: 60%;
}

.progress.flipped {
    transform: scaleX(-1);
}

"""
for color in colors:
    customColors += f""".progress.is-{color.replace(' ','').lower()}::-moz-progress-bar {{
\tbackground-color: {colors[color]};
}}
"""

# Generate yearly reports
for yr in years:
    try:
        df = inDataFrame[inDataFrame.Date.dt.year==yr]

        # Get table text
        catEntryStr = catEntry(df)

        [pN, pP] = yearlyPlot(yr, df, colors)
        totWidth = (pN.width + pP.width)
        p = monthlyPlot(yr, df, colors, totWidth)


        divTitle = Div(text=f"""
    <link rel="stylesheet" href="https://unpkg.com/bulma@0.9.4/css/bulma.min.css" />
    <div class="container">
        <div class="column is-12">
            <h1 class="title has-text-centered section-title">
                {yr} RECAP
            </h1>
        </div>
    </div>
    """)
        divTable = Div(text = f"""  <link rel="stylesheet" href="https://unpkg.com/bulma@0.9.4/css/bulma.min.css" />
    <style type="text/css">
        {customColors}
        .fit {{
            width: {totWidth}px;
            padding: 1.5rem;
        }}
    </style>
    <div class="table-container fit">
        <table class="table is-narrow" width="100%">
            <thead>
                <tr>
                    <th>Category</th>
                    <th>Yearly expenses</th>
                    <th>Yearly revenue</th>
                    <th>Yearly net</th>
                    <th>Monthly average</th>
                    <th>Weekly average</th>
                </tr>
            </thead>
            <tbody>
                    {catEntryStr}
            </tbody>
        </table>
    </div>
    """)
        show(Column(divTitle, Row(pN,pP), p, divTable))
    except Exception as e:
        print(yr,e)
        pass